In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from grouper import group_by_volume, group_by_time
from pyfolio.timeseries import perf_stats
from functools import partial
import sys
sys.path.append('/home/tomek/ib_tools')
from indicators import get_ATR, get_signals
%matplotlib inline

In [2]:
contract = pd.read_pickle('data/minute_NQ_cont_non_active_included.pickle')
multiplier = 20
min_tick = 0.25

In [3]:
contract = contract.loc['20180202':]
contract

,open,high,low,close,volume,average,barCount
date,,,,,,,
2018-02-02 00:00:00,6954.00,6958.00,6953.50,6955.25,877,6955.025,362
2018-02-02 00:01:00,6955.25,6956.50,6952.50,6952.50,376,6955.150,201
2018-02-02 00:02:00,6952.50,6954.00,6951.25,6952.00,292,6952.525,141
2018-02-02 00:03:00,6951.75,6953.50,6951.00,6953.00,414,6952.250,140
2018-02-02 00:04:00,6952.75,6956.50,6952.75,6956.50,304,6954.275,123
...,...,...,...,...,...,...,...
2019-09-04 18:27:00,7695.50,7696.00,7694.75,7695.00,369,7695.475,178
2019-09-04 18:28:00,7695.00,7695.50,7694.75,7695.00,173,7695.025,72
2019-09-04 18:29:00,7695.00,7695.75,7694.50,7695.00,195,7695.125,87


In [4]:
#periods = 20
periods = [5, 10, 20, 40, 80] #, 160, 320]
ema_fast = 120 # number of periods for moving average filter
sl_atr = 1 # stop loss in ATRs
atr_periods = 80 # number of periods to calculate ATR on
time_int = 60 # interval in minutes to be used to define volume candle
commission = 2.1 # level of commision per transaction in dollars
bankroll = 15000

In [5]:
avg_vol = contract.loc['20190621':].volume.rolling(time_int).sum().mean()
avg_vol

19835.00226841541

In [6]:
avg_vol = 14913

In [7]:
vol_candles = group_by_volume(contract, avg_vol)
vol_candles['atr'] = get_ATR(vol_candles, atr_periods)

In [8]:
vol_candles

,date,open,high,low,close,barCount,volume,atr
label,,,,,,,,
0,2018-02-02 02:42:00,6954.00,6963.50,6912.50,6915.00,7971,14857,51.000000
1,2018-02-02 07:59:00,6915.25,6944.00,6915.00,6932.75,8645,14991,39.862500
2,2018-02-02 10:20:00,6932.75,6939.75,6888.00,6889.50,7977,14427,43.924470
3,2018-02-02 12:04:00,6889.75,6889.75,6857.75,6867.25,9138,16018,40.830641
4,2018-02-02 13:34:00,6867.00,6872.50,6844.50,6866.25,8830,15101,38.134623
...,...,...,...,...,...,...,...,...
12030,2019-09-04 16:43:00,7675.75,7683.75,7668.00,7676.50,7942,15511,24.377828
12031,2019-09-04 17:08:00,7676.50,7686.75,7673.00,7684.75,7686,14993,24.115413
12032,2019-09-04 17:32:00,7685.00,7698.75,7679.50,7697.25,7165,15097,23.995279


In [9]:
data = vol_candles
data['ema'] = data.close.ewm(span=ema_fast).mean()
#data['min'] = data.close.rolling(periods).min()
#data['max'] = data.close.rolling(periods).max()
rows_to_drop = max(*periods, ema_fast)
data = data.iloc[rows_to_drop:]
data.reset_index(drop=True, inplace=True)

In [10]:
#data['buy_signal'] = (data.close >= data['max']) * 1
#data['sell_signal'] = (data.close <= data['min']) * -1
#data['signal'] = data.buy_signal + data.sell_signal
#del data['buy_signal']
#del data['sell_signal']
data['signal'] = get_signals(data.close, periods)
data['filter'] = np.sign(data['close'] - data['ema'])
data['filtered_signal'] = data['signal'] * ((data['signal'] * data['filter']) == 1)
#data['cum_position'] = data['position'].cumsum().clip(lower=-1, upper=1)
#data.tail(50)

In [11]:
# while in position maintain open price and transaction direction
data['positions'] = 0
# flag to execute transaction at next data point
data['mark'] = False
# note the reason for transaction at next data point
data['reason'] = ''
# record commission paid
data['commission'] = 0
# record transaction price
data['price'] = 0
# entry price for stop loss calculation
data['entry'] = 0

for item in data.itertuples(): 
    # first row doesn't have to check if we have positions or execute transactions
    if not item.Index == 0:
        # starting position is the same as previous day position
        data.loc[item.Index, 'positions'] = data.loc[(item.Index - 1), 'positions'] 
        data.loc[item.Index, 'entry'] = data.loc[(item.Index - 1), 'entry'] 
        # execute transactions
        if data.loc[(item.Index -  1), 'mark']:
            # close position
            if data.loc[item.Index, 'positions']:
                data.loc[item.Index, 'positions'] = 0
                data.loc[item.Index, 'entry'] = 0
                # record transaction price
                data.loc[item.Index, 'price'] = item.open * np.sign(data.loc[(item.Index - 1), 'entry']) * -1
            # open position
            else:
                data.loc[item.Index, 'positions'] = data.loc[(item.Index - 1), 'signal']
                data.loc[item.Index, 'entry'] = item.open * data.loc[(item.Index - 1), 'signal']
                # record transaction price
                data.loc[item.Index, 'price'] = item.open * data.loc[(item.Index -1), 'signal']
            # record commission paid
            data.loc[item.Index, 'commission'] = commission

    # check for close signal
    if data.loc[item.Index, 'positions'] != 0 and np.sign(item.signal) != 0:
        if np.sign(data.loc[item.Index, 'positions']) != np.sign(item.signal):
            data.loc[item.Index, 'mark'] = True
            data.loc[item.Index, 'reason'] = 'close'
    # check for stop-loss signal
    # long positions
    if data.loc[item.Index, 'positions'] > 0:
        if item.close <= (data.loc[item.Index, 'entry'] - (item.atr * sl_atr)):
            data.loc[item.Index, 'mark'] = True
            data.loc[item.Index, 'reason'] = 'stop-out'
    # short positions
    if data.loc[item.Index, 'positions'] < 0:
        if item.close >= abs((data.loc[item.Index, 'entry'] - (item.atr * sl_atr))):
            data.loc[item.Index, 'mark'] = True
            data.loc[item.Index, 'reason'] = 'stop-out'
    # check for entry signal
    if data.loc[item.Index, 'positions'] == 0:
        if item.filtered_signal != 0:
            data.loc[item.Index, 'mark'] = True
            data.loc[item.Index, 'reason'] = 'entry'
# close any open positions
if data[data.price != 0].price.count() % 2 != 0:
    data.loc[data.index[-1]+1, 'price'] = data.open.iloc[-1] * np.sign(data.entry.iloc[-1]) * -1
    data.loc[data.index[-2]+1, 'entry'] = 0
data.set_index('date', inplace=True)

In [12]:
data.head(50)

,open,high,low,close,barCount,volume,atr,ema,signal,filter,filtered_signal,positions,mark,reason,commission,price,entry
date,,,,,,,,,,,,,,,,,
2018-02-06 15:58:00,6592.50,6599.75,6520.25,6523.50,8414.0,17057.0,54.057955,6607.554781,0.0,-1.0,0.0,0.0,False,,0.0,0.0,0.0
2018-02-06 16:04:00,6523.50,6589.50,6523.50,6586.75,8534.0,15351.0,54.367476,6607.159110,0.0,-1.0,0.0,0.0,False,,0.0,0.0,0.0
2018-02-06 16:12:00,6587.00,6599.75,6536.00,6542.25,9299.0,17108.0,54.610361,6605.927716,0.0,-1.0,0.0,0.0,False,,0.0,0.0,0.0
2018-02-06 16:19:00,6542.75,6548.50,6471.75,6499.50,8969.0,16461.0,55.182802,6603.913590,0.0,-1.0,0.0,0.0,False,,0.0,0.0,0.0
2018-02-06 16:25:00,6499.00,6530.75,6456.25,6483.75,8746.0,15608.0,55.681686,6601.644950,0.0,-1.0,0.0,0.0,False,,0.0,0.0,0.0
2018-02-06 16:33:00,6484.25,6503.00,6452.00,6461.75,9115.0,16138.0,55.560914,6599.009984,0.0,-1.0,0.0,0.0,False,,0.0,0.0,0.0
2018-02-06 16:41:00,6463.00,6473.75,6422.50,6462.00,8536.0,15621.0,55.449830,6596.435296,0.0,-1.0,0.0,0.0,False,,0.0,0.0,0.0
2018-02-06 16:51:00,6463.00,6522.50,6445.00,6495.75,8658.0,15329.0,56.017411,6594.547492,0.0,-1.0,0.0,0.0,False,,0.0,0.0,0.0
2018-02-06 17:09:00,6496.25,6529.50,6463.00,6485.50,9629.0,15700.0,56.286954,6592.507431,0.0,-1.0,0.0,0.0,False,,0.0,0.0,0.0


In [33]:
data.tail(50)

,open,high,low,close,barCount,volume,atr,ema,signal,filter,filtered_signal,positions,mark,reason,commission,price,entry
date,,,,,,,,,,,,,,,,,
2019-09-03 09:07:00,7625.50,7658.75,7600.75,7619.50,9678.0,14839.0,28.083321,7649.603877,0.0,-1.0,0.0,0.0,False,,0.0,0.00,0.00
2019-09-03 10:04:00,7619.75,7624.75,7595.50,7612.75,9621.0,15213.0,28.112128,7648.994722,0.0,-1.0,0.0,0.0,False,,0.0,0.00,0.00
2019-09-03 11:25:00,7612.50,7649.75,7610.50,7631.25,9979.0,15302.0,28.387137,7648.701420,0.0,-1.0,0.0,0.0,False,,0.0,0.00,0.00
2019-09-03 12:49:00,7631.25,7633.00,7593.25,7626.25,9177.0,15161.0,28.667702,7648.330323,0.0,-1.0,0.0,0.0,False,,0.0,0.00,0.00
2019-09-03 14:19:00,7625.75,7640.00,7617.75,7628.50,9425.0,15045.0,28.509240,7648.002549,0.0,-1.0,0.0,0.0,False,,0.0,0.00,0.00
2019-09-03 14:53:00,7628.50,7662.00,7627.00,7649.00,9068.0,14937.0,28.669506,7648.019035,0.0,1.0,0.0,0.0,False,,0.0,0.00,0.00
2019-09-03 15:29:00,7648.75,7659.00,7632.25,7638.00,7781.0,12196.0,28.622110,7647.853432,0.0,-1.0,0.0,0.0,False,,0.0,0.00,0.00
2019-09-03 15:35:00,7638.00,7660.00,7627.75,7657.00,10361.0,16829.0,28.711688,7648.004615,0.0,1.0,0.0,0.0,False,,0.0,0.00,0.00
2019-09-03 15:45:00,7656.75,7669.00,7648.50,7665.25,10454.0,17244.0,28.508930,7648.289662,0.0,1.0,0.0,0.0,False,,0.0,0.00,0.00


In [13]:
pnl = data[['open','positions', 'commission', 'mark', 'reason', 'price', 'entry']]
pnl['mark'] = pnl['mark'].shift(1)
pnl['reason'] = pnl['reason'].shift(1)

In [14]:
pnl

,open,positions,commission,mark,reason,price,entry
date,,,,,,,
2018-02-06 15:58:00,6592.50,0.0,0.0,NaN,NaN,0.00,0.0
2018-02-06 16:04:00,6523.50,0.0,0.0,False,,0.00,0.0
2018-02-06 16:12:00,6587.00,0.0,0.0,False,,0.00,0.0
2018-02-06 16:19:00,6542.75,0.0,0.0,False,,0.00,0.0
2018-02-06 16:25:00,6499.00,0.0,0.0,False,,0.00,0.0
...,...,...,...,...,...,...,...
2019-09-04 17:08:00,7676.50,1.0,0.0,False,,0.00,7655.0
2019-09-04 17:32:00,7685.00,1.0,0.0,False,,0.00,7655.0
2019-09-04 18:03:00,7697.25,1.0,0.0,False,,0.00,7655.0


In [15]:
-pnl.price.sum() * multiplier

34460.0

In [16]:
pnl = pnl[pnl.price != 0]
pnl['close'] = pnl.price.shift(-1)
pnl = pnl[pnl.reason == 'entry']
pnl['pnl'] = -(pnl.entry + pnl.close) * multiplier
pnl['net_pnl'] = pnl.pnl - 2 * pnl.commission
pnl['cum_pnl'] = pnl.pnl.cumsum()
pnl['net_cum_pnl'] = pnl.net_pnl.cumsum()
pnl['balance'] = bankroll + pnl.net_cum_pnl
pnl

,open,positions,commission,mark,reason,price,entry,close,pnl,net_pnl,cum_pnl,net_cum_pnl,balance
date,,,,,,,,,,,,,
2018-02-06 21:12:00,6627.00,1.0,2.1,True,entry,6627.00,6627.00,-6619.00,-160.0,-164.2,-160.0,-164.2,14835.8
2018-02-07 19:29:00,6611.25,-1.0,2.1,True,entry,-6611.25,-6611.25,6353.00,5165.0,5160.8,5005.0,4996.6,19996.6
2018-02-09 21:53:00,6430.25,1.0,2.1,True,entry,6430.25,6430.25,-6488.00,1155.0,1150.8,6160.0,6147.4,21147.4
2018-02-14 16:32:00,6606.25,1.0,2.1,True,entry,6606.25,6606.25,-6788.25,3640.0,3635.8,9800.0,9783.2,24783.2
2018-02-20 17:40:00,6819.25,1.0,2.1,True,entry,6819.25,6819.25,-6792.75,-530.0,-534.2,9270.0,9249.0,24249.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-29 10:24:00,7645.75,1.0,2.1,True,entry,7645.75,7645.75,-7685.00,785.0,780.8,35695.0,34069.6,49069.6
2019-09-02 04:49:00,7630.25,-1.0,2.1,True,entry,-7630.25,-7630.25,7667.25,-740.0,-744.2,34955.0,33325.4,48325.4
2019-09-03 00:18:00,7603.00,-1.0,2.1,True,entry,-7603.00,-7603.00,7631.00,-560.0,-564.2,34395.0,32761.2,47761.2


In [17]:
pnl.pnl.sum()

34460.0

In [18]:
pnl.net_pnl.sum()

32817.8

In [19]:
# percentage of winning transactions
win_pct = round(pnl[pnl.pnl > 0].pnl.count() / pnl.pnl.count(), 4)
win_pct

0.2967

In [20]:
# avg. win,  avg. lose, ratio
avg_win = round(pnl[pnl.pnl > 0].pnl.mean(), 2)
avg_lose = round(pnl[pnl.pnl < 0].pnl.mean(), 2)
ratio = round(abs(avg_win / avg_lose), 2)
avg_win, avg_lose, ratio

(1775.65, -623.69, 2.85)

In [21]:
# transaction EV
ev = win_pct * avg_win + (1 - win_pct) * avg_lose
ev

88.19417799999997

In [22]:
# number of transactions
pnl.pnl.count()

391

In [23]:
pnl.pnl.count() * ev

34483.92359799999

In [24]:
pnl.pnl.min(), pnl.pnl.max(), round(abs(pnl.pnl.max() / pnl.pnl.min()), 1)

(-1685.0, 5545.0, 3.3)

In [25]:
daily = pnl.resample('D').last()
daily['return'] = daily.balance.pct_change()

In [26]:
data.commission.sum()

1640.1000000000001

In [27]:
perf_stats(daily['return'])

Annual return          0.668676
Cumulative returns     2.223136
Annual volatility      0.568602
Sharpe ratio           1.168180
Calmar ratio           1.717111
Stability              0.657795
Max drawdown          -0.389419
Omega ratio            1.378338
Sortino ratio          2.570456
Skew                        NaN
Kurtosis                    NaN
Tail ratio             2.108099
Daily value at risk   -0.069001
dtype: float64

In [28]:
data.price.count()

11916

In [29]:
data[data.price != 0].price.count()

782